### Método 2
Con la misma especie Acyrthosiphon pisum y por ende el mismo gff. 
El genoma tiene 4 cromosomas y 21223 unplaced scaffolds. O sea, 21227 secuencias en formato fasta.
Número de genes: 20.446 (nos dió 20020 en el gff, esta vez no contar ni pseudogenes ni nada más, quedarse con esa cantidad).
Estrategia:
* Extraer filas de "gene"
* group by '+' y '-'
* groupby ID scaffold (1° columna)
* Restar 500000 al inicio y sumar 500000 al final de cada gen (Columnas 4 y 5). Si llega a 0 restar hasta ahí.
* Calcular cuantas filas (genes) hay en cada una de las operaciones. O sea cuantos genes hay 500000 lugares arriba y abajo de c/gen.
* Hacer un promedio entre las 20020 operaciones.

In [1]:
# Importamos librerías
import pandas as pd
import gffpandas.gffpandas as gffpd

In [2]:
# Ruta al archivo GFF
gff_filename = "genomic.gff"
gff_df = gffpd.read_gff3(gff_filename)

In [3]:
# Convertir el Gff3DataFrame a un DataFrame de pandas
df = gff_df.df

In [4]:
df['type'].value_counts()

exon                  253133
CDS                   220444
mRNA                   27917
region                 21228
gene                   20020
cDNA_match              4989
lnc_RNA                 2229
transcript               595
tRNA                     341
pseudogene               297
primary_transcript       123
miRNA                    123
rRNA                     105
snoRNA                    61
snRNA                     54
tandem_repeat              2
Name: type, dtype: int64

---

In [5]:
# Sólo nos quedamos con los datos de tipo 'gene'
data_genes = df[(df['type']=='gene')]

In [6]:
data_genes

,seq_id,source,type,start,end,score,strand,phase,attributes
1,NC_042493.1,Gnomon,gene,6394,8916,.,+,.,ID=gene-LOC100569061;Dbxref=GeneID:100569061;N...
7,NC_042493.1,Gnomon,gene,6860,9850,.,-,.,ID=gene-LOC115033410;Dbxref=GeneID:115033410;N...
13,NC_042493.1,Gnomon,gene,15189,15686,.,+,.,ID=gene-LOC107882353;Dbxref=GeneID:107882353;N...
17,NC_042493.1,Gnomon,gene,105433,127327,.,+,.,ID=gene-LOC107882485;Dbxref=GeneID:107882485;N...
39,NC_042493.1,Gnomon,gene,151002,208448,.,+,.,ID=gene-LOC100166236;Dbxref=APHIDBASE:ACYPI007...
...,...,...,...,...,...,...,...,...,...
551645,NC_011594.1,RefSeq,gene,15802,16779,.,+,.,ID=gene-KEG88_p02;Dbxref=GeneID:7055887;Name=N...
551649,NC_011594.1,RefSeq,gene,16778,16841,.,+,.,ID=gene-KEG88_t20;Dbxref=GeneID:64988016;Name=...
551652,NC_011594.1,RefSeq,gene,16779,16842,.,+,.,ID=gene-KEG88_t21;Dbxref=GeneID:64988030;Name=...
551655,NC_011594.1,RefSeq,gene,16835,16902,.,-,.,ID=gene-KEG88_t22;Dbxref=GeneID:64988031;Name=...


In [7]:
##### Eliminamos del dataframe anterior los genes con ID único

In [8]:
# Obtener el recuento de cada valor
value_counts = data_genes['seq_id'].value_counts()

In [9]:
# Seleccionar los valores donde el recuento es igual a 1
to_remove = value_counts[value_counts == 1].index

In [10]:
# Mantener las filas donde la columna de ID no esté en to_remove
data_genes_2 = data_genes[~data_genes.seq_id.isin(to_remove)]

In [11]:
data_genes_2

,seq_id,source,type,start,end,score,strand,phase,attributes
1,NC_042493.1,Gnomon,gene,6394,8916,.,+,.,ID=gene-LOC100569061;Dbxref=GeneID:100569061;N...
7,NC_042493.1,Gnomon,gene,6860,9850,.,-,.,ID=gene-LOC115033410;Dbxref=GeneID:115033410;N...
13,NC_042493.1,Gnomon,gene,15189,15686,.,+,.,ID=gene-LOC107882353;Dbxref=GeneID:107882353;N...
17,NC_042493.1,Gnomon,gene,105433,127327,.,+,.,ID=gene-LOC107882485;Dbxref=GeneID:107882485;N...
39,NC_042493.1,Gnomon,gene,151002,208448,.,+,.,ID=gene-LOC100166236;Dbxref=APHIDBASE:ACYPI007...
...,...,...,...,...,...,...,...,...,...
551645,NC_011594.1,RefSeq,gene,15802,16779,.,+,.,ID=gene-KEG88_p02;Dbxref=GeneID:7055887;Name=N...
551649,NC_011594.1,RefSeq,gene,16778,16841,.,+,.,ID=gene-KEG88_t20;Dbxref=GeneID:64988016;Name=...
551652,NC_011594.1,RefSeq,gene,16779,16842,.,+,.,ID=gene-KEG88_t21;Dbxref=GeneID:64988030;Name=...
551655,NC_011594.1,RefSeq,gene,16835,16902,.,-,.,ID=gene-KEG88_t22;Dbxref=GeneID:64988031;Name=...


In [12]:
# No se observan valores únicos de IDs
data_genes_2['seq_id'].value_counts()

NC_042494.1       6480
NC_042493.1       4741
NC_042495.1       4591
NC_042496.1       1805
NW_021770374.1      78
                  ... 
NW_021770659.1       2
NW_021770667.1       2
NW_021770684.1       2
NW_021770709.1       2
NW_021770753.1       2
Name: seq_id, Length: 372, dtype: int64

In [13]:
##### Ordenamos el dataframe primero por strand (+,-), luego por 'seq_id, y por último 'start' y 'end'

In [14]:
data_genes_2.sort_values(['strand', 'seq_id','start', 'end'], inplace = True, ascending=True)

/tmp/ipykernel_23685/2720884212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2.sort_values(['strand', 'seq_id','start', 'end'], inplace = True, ascending=True)


In [15]:
##### Reseteamos el índice del dataframe

In [16]:
data_genes_2.reset_index(drop=True, inplace=True)

In [17]:
##### Eliminamos columnas innecesarias

In [18]:
data_genes_2.drop(columns=['source', 'type', 'score', 'phase', 'attributes'], inplace = True)

/tmp/ipykernel_23685/1411996876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2.drop(columns=['source', 'type', 'score', 'phase', 'attributes'], inplace = True)


In [19]:
data_genes_2

,seq_id,start,end,strand
0,NC_011594.1,1,1531,+
1,NC_011594.1,1532,1599,+
2,NC_011594.1,1603,2274,+
3,NC_011594.1,2277,2349,+
4,NC_011594.1,2350,2412,+
...,...,...,...,...
18865,NW_021779252.1,58,654,-
18866,NW_021779808.1,11909,13336,-
18867,NW_021779808.1,17083,18111,-
18868,NW_021779988.1,3650,4824,-


In [20]:
# Ciclo en el que aumentamos y restamos en 500000 el inicio y final del gen

In [21]:
cont = 0
for i,j in zip(data_genes_2['start'],data_genes_2['end']):
    if int(i)-500000 < 0:
        data_genes_2.at[cont,'start_menos_500000']=0
    else:
        data_genes_2.at[cont,'start_menos_500000']=i-500000
    data_genes_2.at[cont,'end_mas_500000']=j+500000
    cont = cont + 1

/tmp/ipykernel_23685/3753583713.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2.at[cont,'start_menos_500000']=0
/tmp/ipykernel_23685/3753583713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2.at[cont,'end_mas_500000']=j+500000


In [22]:
data_genes_2

,seq_id,start,end,strand,start_menos_500000,end_mas_500000
0,NC_011594.1,1,1531,+,0.0,501531.0
1,NC_011594.1,1532,1599,+,0.0,501599.0
2,NC_011594.1,1603,2274,+,0.0,502274.0
3,NC_011594.1,2277,2349,+,0.0,502349.0
4,NC_011594.1,2350,2412,+,0.0,502412.0
...,...,...,...,...,...,...
18865,NW_021779252.1,58,654,-,0.0,500654.0
18866,NW_021779808.1,11909,13336,-,0.0,513336.0
18867,NW_021779808.1,17083,18111,-,0.0,518111.0
18868,NW_021779988.1,3650,4824,-,0.0,504824.0


In [23]:
data_genes_2['start'] = data_genes_2['start'].astype(int)
data_genes_2['end'] = data_genes_2['end'].astype(int)
data_genes_2['start_menos_500000'] = data_genes_2['start_menos_500000'].astype(int)
data_genes_2['end_mas_500000'] = data_genes_2['end_mas_500000'].astype(int)

/tmp/ipykernel_23685/2736334508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2['start'] = data_genes_2['start'].astype(int)
/tmp/ipykernel_23685/2736334508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2['end'] = data_genes_2['end'].astype(int)
/tmp/ipykernel_23685/2736334508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [24]:
data_genes_2

,seq_id,start,end,strand,start_menos_500000,end_mas_500000
0,NC_011594.1,1,1531,+,0,501531
1,NC_011594.1,1532,1599,+,0,501599
2,NC_011594.1,1603,2274,+,0,502274
3,NC_011594.1,2277,2349,+,0,502349
4,NC_011594.1,2350,2412,+,0,502412
...,...,...,...,...,...,...
18865,NW_021779252.1,58,654,-,0,500654
18866,NW_021779808.1,11909,13336,-,0,513336
18867,NW_021779808.1,17083,18111,-,0,518111
18868,NW_021779988.1,3650,4824,-,0,504824


In [25]:
# Función para contar filas que cumplen la condición y actualizar el DataFrame avanzando
def contar_filas_condicion_avanzando(seq_id, strand, end, end_mas_500000, row_idx, total_rows):
    count = 0
    for i in range(row_idx + 1, total_rows):
        if data_genes_2.at[i, 'seq_id'] == seq_id and data_genes_2.at[i, 'strand'] == strand:
            if end <= data_genes_2.at[i, 'end'] <= end_mas_500000:
                count += 1
                data_genes_2.at[i, 'n_filas_cumplen_condicion'] = count
        else:
            return count  # Retorna el conteo si cambia el grupo de seq_id o strand
    return count

# Obtener el número total de filas en el DataFrame
total_rows = len(data_genes_2)

# Inicializar la columna 'n_filas_cumplen_condicion' con ceros
data_genes_2['n_filas_cumplen_condicion'] = 0

# Aplicar la función y agregar una nueva columna
data_genes_2['n_filas_cumplen_condicion'] = data_genes_2.apply(
    lambda x: contar_filas_condicion_avanzando(x['seq_id'], x['strand'], x['end'], x['end_mas_500000'], x.name, total_rows),
    axis=1
)

/tmp/ipykernel_23685/4042986015.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2['n_filas_cumplen_condicion'] = 0
/tmp/ipykernel_23685/4042986015.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2['n_filas_cumplen_condicion'] = data_genes_2.apply(


In [26]:
data_genes_2

,seq_id,start,end,strand,start_menos_500000,end_mas_500000,n_filas_cumplen_condicion
0,NC_011594.1,1,1531,+,0,501531,22
1,NC_011594.1,1532,1599,+,0,501599,21
2,NC_011594.1,1603,2274,+,0,502274,20
3,NC_011594.1,2277,2349,+,0,502349,19
4,NC_011594.1,2350,2412,+,0,502412,18
...,...,...,...,...,...,...,...
18865,NW_021779252.1,58,654,-,0,500654,0
18866,NW_021779808.1,11909,13336,-,0,513336,1
18867,NW_021779808.1,17083,18111,-,0,518111,0
18868,NW_021779988.1,3650,4824,-,0,504824,0


In [27]:
# Función para contar filas que cumplen la condición retrocediendo
def contar_filas_condicion(seq_id, strand, start, start_menos_500000, row_idx):
    count = 0
    for i in range(row_idx - 1, -1, -1):
        if data_genes_2.at[i, 'seq_id'] == seq_id and data_genes_2.at[i, 'strand'] == strand:
            if data_genes_2.at[i, 'start'] >= start_menos_500000 and data_genes_2.at[i, 'start_menos_500000'] <= start:
                count += 1
        else:
            return count  # Retorna el conteo si cambia el grupo de seq_id o strand
    return count

# Aplicar la función y agregar una nueva columna
data_genes_2['n_filas_cumplen_condicion_2'] = data_genes_2.apply(lambda x: contar_filas_condicion(x['seq_id'], x['strand'], x['start'], x['start_menos_500000'], x.name), axis=1)

/tmp/ipykernel_23685/1812261375.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2['n_filas_cumplen_condicion_2'] = data_genes_2.apply(lambda x: contar_filas_condicion(x['seq_id'], x['strand'], x['start'], x['start_menos_500000'], x.name), axis=1)


In [32]:
data_genes_2

,seq_id,start,end,strand,start_menos_500000,end_mas_500000,n_filas_cumplen_condicion,n_filas_cumplen_condicion_2,suma
0,NC_011594.1,1,1531,+,0,501531,22,0,22
1,NC_011594.1,1532,1599,+,0,501599,21,1,22
2,NC_011594.1,1603,2274,+,0,502274,20,2,22
3,NC_011594.1,2277,2349,+,0,502349,19,3,22
4,NC_011594.1,2350,2412,+,0,502412,18,4,22
...,...,...,...,...,...,...,...,...,...
18865,NW_021779252.1,58,654,-,0,500654,0,0,0
18866,NW_021779808.1,11909,13336,-,0,513336,1,0,1
18867,NW_021779808.1,17083,18111,-,0,518111,0,1,1
18868,NW_021779988.1,3650,4824,-,0,504824,0,0,0


In [34]:
data_genes_2['suma'] = data_genes_2['n_filas_cumplen_condicion'] + data_genes_2['n_filas_cumplen_condicion_2']

/tmp/ipykernel_23685/3653689681.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genes_2['suma'] = data_genes_2['n_filas_cumplen_condicion'] + data_genes_2['n_filas_cumplen_condicion_2']


In [35]:
data_genes_2

,seq_id,start,end,strand,start_menos_500000,end_mas_500000,n_filas_cumplen_condicion,n_filas_cumplen_condicion_2,suma
0,NC_011594.1,1,1531,+,0,501531,22,0,22
1,NC_011594.1,1532,1599,+,0,501599,21,1,22
2,NC_011594.1,1603,2274,+,0,502274,20,2,22
3,NC_011594.1,2277,2349,+,0,502349,19,3,22
4,NC_011594.1,2350,2412,+,0,502412,18,4,22
...,...,...,...,...,...,...,...,...,...
18865,NW_021779252.1,58,654,-,0,500654,0,0,0
18866,NW_021779808.1,11909,13336,-,0,513336,1,0,1
18867,NW_021779808.1,17083,18111,-,0,518111,0,1,1
18868,NW_021779988.1,3650,4824,-,0,504824,0,0,0


In [36]:
data_genes_2['suma'].mean()

19.665182829888714

In [37]:
print("GeneDistance:", 1000000/data_genes_2['suma'].mean())

GeneDistance: 50851.29432308761


In [29]:
# data_genes_2.to_excel('data_genes_2_algo.xlsx', index = False)